In [1]:
import pandas as pd 
import pandas as np 
import matplotlib.pyplot

In [2]:
df=pd.read_csv("DatasetExos2.csv" , delimiter=";")
df.columns

Index(['ep (ms)', 'Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z',
       'ID', 'Label', 'Category', 'Set'],
      dtype='object')

1. drop the first 7 columns

In [3]:
df = df.drop(columns=df.columns[:7])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9009 entries, 0 to 9008
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        9007 non-null   object 
 1   Label     9009 non-null   object 
 2   Category  9005 non-null   object 
 3   Set       9003 non-null   float64
dtypes: float64(1), object(3)
memory usage: 281.7+ KB


1.2 removing the null values 

In [4]:
df = df.dropna()
df

,ID,Label,Category,Set
0,B,bench,heavy,30.0
1,B,bench,heavy,30.0
2,B,bench,heavy,30.0
3,B,bench,heavy,30.0
4,B,bench,heavy,30.0
...,...,...,...,...
9004,E,row,medium,40.0
9005,E,row,medium,40.0
9006,E,row,medium,40.0
9007,E,row,medium,40.0


In [5]:
df = df[df["Set"]>0]
df["Set"].value_counts()

Set
51.0      200
23.0      200
19.0      200
1.0       175
55.0      174
         ... 
91.0       43
8.0        42
65.0       42
20.0       38
2000.0      1
Name: count, Length: 92, dtype: int64

In [6]:
Q1 = df["Set"].quantile(0.25)
Q3 = df["Set"].quantile(0.75)

intervals = [min(df["Set"]) , Q1 , Q3 , max(df["Set"])]
labels = ["Low" , "Mid" , "High"]
df["Set"] = pd.cut(df["Set"] , intervals , labels=labels)
df

C:\Users\abdo7\AppData\Local\Temp\ipykernel_8916\1302458868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Set"] = pd.cut(df["Set"] , intervals , labels=labels)


,ID,Label,Category,Set
0,B,bench,heavy,Mid
1,B,bench,heavy,Mid
2,B,bench,heavy,Mid
3,B,bench,heavy,Mid
4,B,bench,heavy,Mid
...,...,...,...,...
9004,E,row,medium,Mid
9005,E,row,medium,Mid
9006,E,row,medium,Mid
9007,E,row,medium,Mid


2. transaction are the rows for example : 
{B , bench , heavy , mid}
and the items are the values of each column in one item Set :

In [7]:
itemset =[]
for i in df.columns :
    for j in df[i].unique():
        itemset.append(j)
print(itemset)

print("item length : "+ str(len(itemset)))
print("transaction number : "+ str(len(df)))

['B', 'A', 'E', 'C', 'D', 'Z', 'bench', 'ohp', 'squat', 'dead', 'row', 'rest', 'raw', 'heavy', 'medium', 'sitting', 'standing', 'Mid', 'High', 'Low', nan]
item length : 21
transaction number : 8996


4. Build DatasetExos_2

In [8]:
df["Set"].value_counts()

Set
Mid     4411
Low     2245
High    2165
Name: count, dtype: int64

In [9]:
df.to_csv("ExoDataset_2.csv")

# PART 2 

In [10]:
def condidat_Gen(k , itemSet=None):
    if k==1:
        new_itemset = itemset
    else :
        new_itemset =[]
        for i in range(len(itemSet)-1):
            for j in range(i+1 , len(itemSet)): 
                if str(itemSet[i]) != "nan" and str(itemSet[j]) !="nan":
                    if type(itemSet[i]) == list: 
                        new_itemset.append(list(set(itemSet[i]) | set(itemSet[j])))
                    else :
                        new_itemset.append([itemSet[i],itemSet[j]])
    return new_itemset

In [11]:
def compute_Support(df , itemSet):
    condidat_support = {}
    if type(itemSet[0])!=list : 
        for i in itemSet:
            if i in df["ID"].unique():
                new_df = df[df["ID"]==i]

            elif i in df["Label"].unique():
                new_df = df[df["Label"]==i]

            elif i in df["Category"].unique():
                new_df = df[df["Category"]==i]

            elif i in df["Set"].unique():
                new_df = df[df["Set"]==i]
            condidat_support[i]=len(new_df)/len(df)
    else : 
        for j in itemSet:
            new_df=df.copy()
            for i in j : 
                if i in df["ID"].unique():
                    new_df = new_df[new_df["ID"]==i]

                elif i in df["Label"].unique():
                    new_df = new_df[new_df["Label"]==i]

                elif i in df["Category"].unique():
                    new_df = new_df[new_df["Category"]==i]

                elif i in df["Set"].unique():
                    new_df = new_df[new_df["Set"]==i]
            condidat_support[str(j)]=len(new_df)/len(df)
    return condidat_support

In [12]:
import ast
def remove_minSup(minSup , Support , itemset):
    for key in Support : 
        if Support[key] < minSup : 
            try : 
                itemset.remove(ast.literal_eval(key))
            except Exception as e:
                itemset.remove(key)
    return itemset

In [13]:
min_sup =0.13
itemCk = condidat_Gen(1)
SupportCk = compute_Support(df , itemCk)
Lk = remove_minSup(min_sup , SupportCk , itemCk)
print(Lk)

['A', 'E', 'C', 'bench', 'ohp', 'squat', 'dead', 'row', 'heavy', 'medium', 'Mid', 'High', 'Low']


In [14]:
result =Lk
while(len(Lk)>1):
    itemCk = condidat_Gen(2 , Lk)
    SupportCk = compute_Support(df , itemCk)
    Lk = remove_minSup(min_sup , SupportCk , itemCk)
    result +=Lk
    print(len(Lk))

result

6
2
0


['A',
 'E',
 'C',
 'bench',
 'ohp',
 'squat',
 'dead',
 'row',
 'heavy',
 'medium',
 'Mid',
 'High',
 'Low',
 ['A', 'Mid'],
 ['E', 'medium'],
 ['E', 'Mid'],
 ['dead', 'medium'],
 ['heavy', 'Mid'],
 ['medium', 'Mid'],
 ['E', 'Mid', 'medium'],
 ['E', 'Mid', 'medium']]